<a href="https://colab.research.google.com/github/Chun-Ping-Wang/data-course-sample/blob/main/homework_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [11]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [12]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-26 11:31:32--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.1’

All_Beauty.csv.1    100%[===================>]  14.78M  19.9MB/s    in 0.7s    

2021-12-26 11:31:33 (19.9 MB/s) - ‘All_Beauty.csv.1’ saved [15499476/15499476]

--2021-12-26 11:31:33--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.1’

meta_All_Beauty.jso 100%[===================>]   9.85M  15.1MB/s    in 0.7s   

In [ ]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [ ]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [ ]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [ ]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')
ratings.head()

,asin,reviewerID,overall,unixReviewTime,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,1407628800,2014-08-10
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000,2013-03-11
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200,2011-12-25


## 資料切分

In [ ]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID')
print(ratings_testings)
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
print(ratings_testings_by_user)
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
print(ratings_testings_by_user)

users = list(ratings_testings_by_user.keys())
print(users)

              asin      reviewerID  overall  unixReviewTime       DATE
234795  B01CJNZKZK  A38KP6TI3BUIM6      1.0      1536192000 2018-09-06
234796  B01CJNZKZK  A2H7VSYLKGKLLJ      1.0      1535932800 2018-09-03
234797  B01CJNZKZK  A1119JJ37ZLB8R      5.0      1535932800 2018-09-03
240438  B01DKQAXC0  A1FKUBRPLG6Y8I      5.0      1537401600 2018-09-20
240439  B01DKQAXC0  A2WX2YLLHJDLX7      5.0      1537315200 2018-09-19
...            ...             ...      ...             ...        ...
371154  B01HFWIKH0  A3UAJ5O4VR3IQ3      5.0      1536710400 2018-09-12
371173  B01HGJHL8G  A3254BNN69CL47      5.0      1536105600 2018-09-05
371246  B01HHWQX8G  A32D5I46E15TEC      3.0      1538265600 2018-09-30
371247  B01HHWQX8G   A1S3NMNTSO6GW      4.0      1537142400 2018-09-17
371252  B01HHUDO7Q  A38GODWWMRN5IP      5.0      1537747200 2018-09-24

[590 rows x 5 columns]
[{'reviewerID': 'A100XQFWKQ30O2', 'asin': ['B01DKQAXC0']}, {'reviewerID': 'A103T1QOGFCSEH', 'asin': ['B01DKQAXC0']}, {'revie

## 產生推薦

In [15]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    Your Code
    '''
    # ___solution1 統一推薦rank top 10 : 0.005084745762711864(k=2/k=5/k=10)
    # product = metadata[['asin','rank']]
    # product['rank'] = product['rank'].str.split(expand=True)[0]
    # product['rank'] = product['rank'].str.replace(',', '')
    # product['rank']  = pd.to_numeric(product['rank'], errors='coerce').astype(np.int64,errors='ignore')
    # top_10_df = product.sort_values('rank').head(10)
    # top_10_product = top_10_df['asin']
    # print(top_10_product.tolist())

    # recommendations = {user: top_10_product.tolist() for user in users}
    # return recommendations

    # ___solution2 rank top 10 && overall : 
    product = metadata[['asin','rank']]
    product['rank'] = product['rank'].str.split(expand=True)[0]
    product['rank'] = product['rank'].str.replace(',', '')
    product['rank']  = pd.to_numeric(product['rank'], errors='coerce').astype(np.int64,errors='ignore')
    mix_df = ratings.set_index('asin').join(product.set_index('asin'))
    mean_df = mix_df.groupby('asin').mean().reset_index()
    df_a = mean_df[mean_df.overall >= 3].sort_values('rank').head(10) #滿意度3以上 && 排行前10 :0.005084745762711864(k=2/k=5/k=10)
    df_b = mean_df[mean_df.overall >= 4].sort_values('rank').head(10) #滿意度4以上 && 排行前10 :0.010169491525423728(k=2/k=5/k=10)
    df_c = mean_df[mean_df.overall >= 4.5].sort_values('rank').head(10) #滿意度4.5以上 && 排行前10 :0.010169491525423728(k=2/k=5/k=10)
    df_d = mean_df[mean_df.overall >= 4.8].sort_values('rank').head(10) #滿意度4.8以上 && 排行前10 :0.005084745762711864(k=2/k=5/k=10)

    recommendations = {user: df_d['asin'] for user in users}
    return recommendations


ratings_by_user = recommender(ratings_trainings, users)

ratings_by_user



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

{'A100XQFWKQ30O2': 8235     B00BXPVIT8
 10531    B00G40FJH4
 985      B000FZ50MS
 27288    B01B16B58C
 3552     B0027BG0YK
 4992     B004RRG984
 29750    B01DSDPEZ6
 27887    B01BRJ8TF0
 15554    B00MXENWO2
 27865    B01BQR7TK4
 Name: asin, dtype: object, 'A103T1QOGFCSEH': 8235     B00BXPVIT8
 10531    B00G40FJH4
 985      B000FZ50MS
 27288    B01B16B58C
 3552     B0027BG0YK
 4992     B004RRG984
 29750    B01DSDPEZ6
 27887    B01BRJ8TF0
 15554    B00MXENWO2
 27865    B01BQR7TK4
 Name: asin, dtype: object, 'A106UKKSJ2KXPF': 8235     B00BXPVIT8
 10531    B00G40FJH4
 985      B000FZ50MS
 27288    B01B16B58C
 3552     B0027BG0YK
 4992     B004RRG984
 29750    B01DSDPEZ6
 27887    B01BRJ8TF0
 15554    B00MXENWO2
 27865    B01BQR7TK4
 Name: asin, dtype: object, 'A10A7GV4D5A11V': 8235     B00BXPVIT8
 10531    B00G40FJH4
 985      B000FZ50MS
 27288    B01B16B58C
 3552     B0027BG0YK
 4992     B004RRG984
 29750    B01DSDPEZ6
 27887    B01BRJ8TF0
 15554    B00MXENWO2
 27865    B01BQR7TK4
 Name: 

## 結果評估

In [16]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)


0.005084745762711864